In [1]:
!pip install -q transformers accelerate torch sentence-transformers yt-dlp opencv-python pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 84.6 MB/s eta 0:00:00


In [2]:
import os
import cv2
import torch
import yt_dlp
import pandas as pd
from PIL import Image
from transformers import BlipProcessor, BlipForQuestionAnswering
from sentence_transformers import SentenceTransformer, util

In [3]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
blip_model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to("cuda" if torch.cuda.is_available() else "cpu")
embedder = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
def download_video(youtube_url, output_path="video"):
    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',
        'outtmpl': f'{output_path}.%(ext)s'
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])
    for ext in ["webm", "mp4"]:
        if os.path.exists(f"{output_path}.{ext}"):
            return f"{output_path}.{ext}"
    return None

def extract_frame(video_path, frame_number=180, output_img="frame.jpg"):
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()
    cap.release()
    if ret:
        cv2.imwrite(output_img, frame)
        return output_img
    return None

def ask_question(image_path, question):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(image, question, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
    output = blip_model.generate(**inputs)
    return processor.decode(output[0], skip_special_tokens=True)

def match_answer_to_option(answer, options):
    answer_vec = embedder.encode(answer, convert_to_tensor=True)
    option_vecs = embedder.encode(options, convert_to_tensor=True)
    scores = util.cos_sim(answer_vec, option_vecs)[0]
    best_idx = int(scores.argmax())
    best_score = float(scores[best_idx])
    return best_idx, best_score

In [5]:
csv_path = "mcq_cleaned.csv"
df = pd.read_csv(csv_path)
results = []

In [6]:
for idx, row in df.iterrows():
    print(f"Processing QID: {row['qid']}")
    qid = row['qid']
    question = row['question']
    url = row['youtube_url']
    options = [row['A'], row['B'], row['C'], row['D'], row.get('E', 'None of the above')]

    try:
        video_path = download_video(url, output_path=f"video_{qid}")
        frame_path = extract_frame(video_path, frame_number=180, output_img=f"frame_{qid}.jpg")
        model_answer = ask_question(frame_path, question)
        best_idx, best_score = match_answer_to_option(model_answer, options)

        results.append({
            "qid": qid,
            "pred": chr(65 + best_idx)
        })

        print(f"Answer: {model_answer} → Option {chr(65 + best_idx)}")

    except Exception as e:
        print(f"Failed for QID {qid}:", e)

Processing QID: 0008-7
[youtube] Extracting URL: https://www.youtube.com/shorts/sj81PWrerDk
[youtube] sj81PWrerDk: Downloading webpage
[youtube] sj81PWrerDk: Downloading tv client config
[youtube] sj81PWrerDk: Downloading player 69f581a5
[youtube] sj81PWrerDk: Downloading tv player API JSON
[youtube] sj81PWrerDk: Downloading ios player API JSON
[youtube] sj81PWrerDk: Downloading m3u8 information
[info] sj81PWrerDk: Downloading 1 format(s): 616+251
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 2
[download] Destination: video_0008-7.f616.mp4
[download] 100% of    3.05MiB in 00:00:00 at 4.73MiB/s                 
[download] Destination: video_0008-7.f251.webm
[download] 100% of  130.24KiB in 00:00:00 at 915.17KiB/s 
[Merger] Merging formats into "video_0008-7.webm"
Deleting original file video_0008-7.f616.mp4 (pass -k to keep)
Deleting original file video_0008-7.f251.webm (pass -k to keep)
Answer: with your hands → Option D
Processing QID: 0012-7
[youtube] Extracting 

ERROR: [youtube] AGCyLqLuUJ0: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.


Failed for QID 0012-7: ERROR: [youtube] AGCyLqLuUJ0: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.
Processing QID: 0016-7
[youtube] Extracting URL: https://www.youtube.com/shorts/4mKxze5P9gQ
[youtube] 4mKxze5P9gQ: Downloading webpage
[youtube] 4mKxze5P9gQ: Downloading tv client config
[youtube] 4mKxze5P9gQ: Downloading player 69f581a5
[youtube] 4mKxze5P9gQ: Downloading tv player API JSON
[youtube] 4mKxze5P9gQ: Downloading ios player API JSON
[youtube] 4mKxze5P9gQ: Downloading m3u8 information
[info] 4mKxze5P9gQ: Downloading 1 format(s): 247+251
[download] Destination: video_0016-7.f247.webm
[download] 100% of  193.94KiB in 00:00:00 at 297.40KiB/s 
[download] Destination: video_0016-7.f251.webm
[download] 100% of  241.05KiB in 00:00:00 at 520.55KiB/s 
[Merger] Merging formats into "video_0016-7.webm"
Deleting original file video_0016-7.f247.webm (pass -k to keep)
Deleting original file video_0016-7.f251.w

ERROR: [youtube] u2HvKEGq3Ik: Video unavailable


Failed for QID 0133-7: ERROR: [youtube] u2HvKEGq3Ik: Video unavailable
Processing QID: 0142-7
[youtube] Extracting URL: https://youtube.com/shorts/rCW1aGpFrJg
[youtube] rCW1aGpFrJg: Downloading webpage
[youtube] rCW1aGpFrJg: Downloading tv client config
[youtube] rCW1aGpFrJg: Downloading player 69f581a5
[youtube] rCW1aGpFrJg: Downloading tv player API JSON
[youtube] rCW1aGpFrJg: Downloading ios player API JSON
[youtube] rCW1aGpFrJg: Downloading m3u8 information
[info] rCW1aGpFrJg: Downloading 1 format(s): 616+251
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 10
[download] Destination: video_0142-7.f616.mp4
[download] 100% of   21.58MiB in 00:00:01 at 15.51MiB/s                
[download] Destination: video_0142-7.f251.webm
[download] 100% of  919.77KiB in 00:00:00 at 2.53MiB/s   
[Merger] Merging formats into "video_0142-7.webm"
Deleting original file video_0142-7.f616.mp4 (pass -k to keep)
Deleting original file video_0142-7.f251.webm (pass -k to keep)
Answer: cau

...

Failed for QID 0833-7: ERROR: [youtube] E2ILLN8TiPA: Video unavailable. This video is no longer available because the YouTube account associated with this video has been terminated.
Processing QID: 0835-7
[youtube] Extracting URL: https://www.youtube.com/shorts/_tVCT7lNvM4
[youtube] _tVCT7lNvM4: Downloading webpage
[youtube] _tVCT7lNvM4: Downloading tv client config
[youtube] _tVCT7lNvM4: Downloading player 69f581a5
[youtube] _tVCT7lNvM4: Downloading tv player API JSON
[youtube] _tVCT7lNvM4: Downloading ios player API JSON
[youtube] _tVCT7lNvM4: Downloading m3u8 information
[info] _tVCT7lNvM4: Downloading 1 format(s): 248+251
[download] Destination: video_0835-7.f248.webm
[download] 100% of    9.31MiB in 00:00:00 at 23.47MiB/s  
[download] Destination: video_0835-7.f251.webm
[download] 100% of  668.20KiB in 00:00:00 at 1.92MiB/s   
[Merger] Merging formats into "video_0835-7.webm"
Deleting original file video_0835-7.f251.webm (pass -k to keep)
Deleting original file video_0835-7.f248.w

ERROR: [youtube] XCkJcIeGhS0: Video unavailable


Failed for QID 0868-7: ERROR: [youtube] XCkJcIeGhS0: Video unavailable
Processing QID: 0870-7
[youtube] Extracting URL: https://www.youtube.com/shorts/Po_UemYmEyg
[youtube] Po_UemYmEyg: Downloading webpage
[youtube] Po_UemYmEyg: Downloading tv client config
[youtube] Po_UemYmEyg: Downloading player 69f581a5
[youtube] Po_UemYmEyg: Downloading tv player API JSON
[youtube] Po_UemYmEyg: Downloading ios player API JSON
[youtube] Po_UemYmEyg: Downloading m3u8 information
[info] Po_UemYmEyg: Downloading 1 format(s): 399+251
[download] Destination: video_0870-7.f399.mp4
[download] 100% of   22.00MiB in 00:00:00 at 34.50MiB/s  
[download] Destination: video_0870-7.f251.webm
[download] 100% of    1.02MiB in 00:00:00 at 14.58MiB/s  
[Merger] Merging formats into "video_0870-7.webm"
Deleting original file video_0870-7.f251.webm (pass -k to keep)
Deleting original file video_0870-7.f399.mp4 (pass -k to keep)
Failed for QID 0870-7: 'NoneType' object has no attribute 'read'
Processing QID: 0871-7
[yo

ERROR: [youtube] 6qHXenlRDH4: Video unavailable. This video is no longer available due to a privacy claim by a third party


Failed for QID 1321-7: ERROR: [youtube] 6qHXenlRDH4: Video unavailable. This video is no longer available due to a privacy claim by a third party
Processing QID: 1324-7
[youtube] Extracting URL: https://www.youtube.com/shorts/L9dnb-wfsnc
[youtube] L9dnb-wfsnc: Downloading webpage
[youtube] L9dnb-wfsnc: Downloading tv client config
[youtube] L9dnb-wfsnc: Downloading player 69f581a5
[youtube] L9dnb-wfsnc: Downloading tv player API JSON
[youtube] L9dnb-wfsnc: Downloading ios player API JSON
[youtube] L9dnb-wfsnc: Downloading m3u8 information
[info] L9dnb-wfsnc: Downloading 1 format(s): 399+251
[download] Destination: video_1324-7.f399.mp4
[download] 100% of    9.92MiB in 00:00:00 at 21.78MiB/s  
[download] Destination: video_1324-7.f251.webm
[download] 100% of  389.67KiB in 00:00:00 at 1.39MiB/s   
[Merger] Merging formats into "video_1324-7.webm"
Deleting original file video_1324-7.f251.webm (pass -k to keep)
Deleting original file video_1324-7.f399.mp4 (pass -k to keep)
Failed for QID 1

In [7]:
results_df = pd.DataFrame(results)
results_df.to_csv("mcq_results.csv", index=False)
print("Saved: mcq_results.csv")
results_df.head()

Saved: mcq_results.csv


,qid,best_option
0,0008-7,D
1,0016-7,A
2,0036-7,D
3,0037-7,D
4,0038-7,D
